In [4]:
import numpy as np
import pandas as pd
import requests as rq
import time
import urllib.parse as urlhandler

In [3]:
fs_data = pd.read_csv("data/Fire_Safety_Deficiencies.csv")
fs_data['Survey Date'] = pd.to_datetime(fs_data['Survey Date'])
fs_data['Correction Date'] = pd.to_datetime(fs_data['Correction Date'])
fs_data = fs_data[fs_data['Survey Date'].dt.year > 2014]
c1 = fs_data['Location'].str.split("(", 1, True).iloc[:,1]
c2 = c1.str.split(")", 1, True).iloc[:,0]
coordinates = c2.str.split(", ", 1, True)
fs_data = fs_data.assign(Lat=coordinates[0].values)
fs_data = fs_data.assign(Long=coordinates[1].values)
fs_data['Full Address'] = fs_data['Provider Address'] + ' ' + fs_data['Provider City'] + ', ' + fs_data['Provider State'] + ' ' + fs_data['Provider Zip Code'].astype(str)
fs_data = fs_data.drop(columns=['Location'])
nullloc = fs_data[fs_data['Lat'].isna()]
nullloc.head(3)

,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Survey Date,Survey Type,Deficiency Prefix,Deficiency Category,...,Scope Severity Code,Deficiency Corrected,Correction Date,Inspection Cycle,Standard Deficiency,Complaint Deficiency,Processing Date,Lat,Long,Full Address
0,366402,WILLOW RIDGE OF MENNONITE HOME COMMUNITIES OF ...,101 WILLOW RIDGE DRIVE,BLUFFTON,OH,45817,2019-05-01,Fire Safety,K,Smoke Deficiencies,...,F,"Deficient, Provider has date of correction",2019-06-20,1,True,N,04/01/2020,NaN,NaN,"101 WILLOW RIDGE DRIVE BLUFFTON, OH 45817"
1,015075,SUMMERFORD NURSING HOME INC,4087 HIGHWAY 31 SOUTHWEST,FALKVILLE,AL,35622,2017-09-27,Fire Safety,K,Smoke Deficiencies,...,D,"Deficient, Provider has date of correction",2017-10-09,3,True,N,04/01/2020,NaN,NaN,"4087 HIGHWAY 31 SOUTHWEST FALKVILLE, AL 35622"
3,015124,CLAY COUNTY NURSING HOME,83825 HIGHWAY 9 P O BOX 1270,ASHLAND,AL,36251,2017-05-17,Fire Safety,K,Smoke Deficiencies,...,D,"Deficient, Provider has date of correction",2017-06-21,3,True,N,04/01/2020,NaN,NaN,"83825 HIGHWAY 9 P O BOX 1270 ASHLAND, AL 36251"


In [41]:
def get_lat_lng(addr):
    f=open("data/googleapikey.txt","r");key=f.read();f.close();
    url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    query = {
        'address': urlhandler.quote(addr),
        'key': key
    };key=None;
    time.sleep(5)
    response = rq.get(url, query);query=None;
    if response.ok:
        rawjson = response.json()
        data = pd.json_normalize(rawjson['results']).reset_index(drop=True)
        lat = data['geometry.location.lat'][0]
        lng = data['geometry.location.lng'][0]
        print(f"location: {addr}")
        return lat, lng
    return np.nan, np.nan

In [43]:
data

,access_points,address_components,formatted_address,partial_match,place_id,types,geometry.bounds.northeast.lat,geometry.bounds.northeast.lng,geometry.bounds.southwest.lat,geometry.bounds.southwest.lng,geometry.location.lat,geometry.location.lng,geometry.location_type,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng
0,[],"[{'long_name': 'Falkville', 'short_name': 'Fal...","Falkville, AL, USA",True,ChIJJTe3H6ksiIgRHwXY9aGGR5Y,"[locality, political]",34.404319,-86.886087,34.362979,-86.926154,34.368429,-86.908616,APPROXIMATE,34.404319,-86.886087,34.362979,-86.926154


In [42]:
for loc in nullloc['Full Address'].unique():
    lat, lng = get_lat_lng(loc)
    nullloc[nullloc['Full Address'] == loc]['Lat'] = lat
    nullloc[nullloc['Full Address'] == loc]['Long'] = lng

location: 101 WILLOW RIDGE DRIVE BLUFFTON, OH 45817


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


location: 4087 HIGHWAY 31 SOUTHWEST FALKVILLE, AL 35622
location: 83825 HIGHWAY 9   P O BOX 1270 ASHLAND, AL 36251
location: THIRD AVENUE    P O BOX 607 MOUNDVILLE, AL 35474


KeyError: 'geometry.location.lat'

In [44]:
nullloc

,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Survey Date,Survey Type,Deficiency Prefix,Deficiency Category,...,Scope Severity Code,Deficiency Corrected,Correction Date,Inspection Cycle,Standard Deficiency,Complaint Deficiency,Processing Date,Lat,Long,Full Address
0,366402,WILLOW RIDGE OF MENNONITE HOME COMMUNITIES OF ...,101 WILLOW RIDGE DRIVE,BLUFFTON,OH,45817,2019-05-01,Fire Safety,K,Smoke Deficiencies,...,F,"Deficient, Provider has date of correction",2019-06-20,1,True,N,04/01/2020,NaN,NaN,"101 WILLOW RIDGE DRIVE BLUFFTON, OH 45817"
1,015075,SUMMERFORD NURSING HOME INC,4087 HIGHWAY 31 SOUTHWEST,FALKVILLE,AL,35622,2017-09-27,Fire Safety,K,Smoke Deficiencies,...,D,"Deficient, Provider has date of correction",2017-10-09,3,True,N,04/01/2020,NaN,NaN,"4087 HIGHWAY 31 SOUTHWEST FALKVILLE, AL 35622"
3,015124,CLAY COUNTY NURSING HOME,83825 HIGHWAY 9 P O BOX 1270,ASHLAND,AL,36251,2017-05-17,Fire Safety,K,Smoke Deficiencies,...,D,"Deficient, Provider has date of correction",2017-06-21,3,True,N,04/01/2020,NaN,NaN,"83825 HIGHWAY 9 P O BOX 1270 ASHLAND, AL 36251"


In [ ]:
# US CENSUS API FOR DEMOGRAPHICS
# f=open("data/uscensusapikey.txt","r");key=f.read()



In [ ]:
# GOOGLE API FOR GEOCODING
# https://console.cloud.google.com/google/maps-apis/overview;onboard=true?project=vaulted-keel-277804
# https://developers.google.com/maps/documentation/geocoding/intro#BYB
f=open("data/googleapikey.txt","r");key=f.read();f.close();key=None